In [1]:
from portfolio import Portfolio
from asset import Asset
from pandas import Timestamp as tmpstemp
from pandas import Timedelta as tmpdelta
import pandas as pd
import os
from coinbase.rest import RESTClient
import json
import uuid

In [2]:
os.environ['MY_ENVIRONMENT'] = 'prod'
if 'API_KEY' in os.environ:
    client = RESTClient(api_key = os.environ['API_KEY'], api_secret=os.environ['API_SECRET'])
else:
    try:
        with open('coinbase_cloud_api_key.json') as f:
            d = json.load(f)
        os.environ['API_KEY'] =d['name']
        os.environ['API_SECRET'] = d['privateKey']
        client = RESTClient(api_key = os.environ['API_KEY'], api_secret=os.environ['API_SECRET'])
    except:
        print('Can not find keys')

In [3]:
BTC = Asset('BTC')
ETH = Asset('ETH')
USDT = Asset('USDT')
rebal_portfolio = Portfolio({'BTC':0, 'ETH':0, 'USDT':0})

In [7]:
order_id = str(uuid.uuid4())
trans_type = 'BUY'
base_size = 0.01
quote_size = 15
trade_pair = 'ETH-USDT'
to_drop = 'whatever'
if trans_type  == 'SELL':
    print(f'Sell {base_size} of {trade_pair} and  drop {to_drop}, id is {order_id }')

    order = client.market_order_sell(
            client_order_id=order_id,
            product_id=trade_pair,
                base_size=str(base_size)
            )
    fills = client.get_fills(order_id=order_id)
    print(fills)
else:
    print(f'Buy {quote_size} of {trade_pair} and  drop {to_drop}, id is {order_id }')

    order = client.market_order_buy(
            client_order_id=order_id,
            product_id=trade_pair,
                quote_size=str(quote_size)
            )
    fills = client.get_fills(order_id=order_id)
    print(fills)

Buy 15 of ETH-USDT and  drop whatever, id is 93bf9f33-b33c-4181-8b17-1a72bc10c551
{'fills': [], 'cursor': ''}


In [8]:
positions = rebal_portfolio.get_current_postions()
for ticker in positions.index:
    alloc = positions['allocation'].loc[ticker] *100
    print(f"{ticker}: {round(alloc,1)}%")

BTC: 39.6%
ETH: 19.9%
USDT: 40.5%


In [14]:
len(client.get_product('ETH-USDT')['base_increment'].split('.')[1])

8